In [ ]:
! pip install --user --upgrade requests bs4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://readwrite.com/" #해외 IT 전문 미디어
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

In [ ]:
# category별 url 뽑기
categories = dict()
not_more = soup.find('div', {"class": "menu-main-menu-container"}).find_all('a') # 더보기를 안눌러도 볼 수 있는 categories
for a in not_more:
  if 'category' in a["href"]:
    #categories.append((a.text, a["href"]))
    categories[a.text] = a["href"]

more = soup.find_all("div", {"class": "col-md-12"})[:2] # 더보기를 누르면 볼 수 있는 categories
for div in more:
  a_list = div.find_all('a')
  for a in a_list:
    if 'category' in a["href"]:
      #categories.append((a.text, a["href"]))
      category_name = a.text; category_url = a["href"]
      if '/' in category_name:
        category_name = category_name.replace('/', '_')
      categories[category_name] = category_url
#####
print(categories)
# categories_list = list(categories)[:18]
# for c in categories_list:
#   del(categories[c])
# print(categories)
#####

In [ ]:
# html 파일 크롤링

from google.colab import drive
import time

drive.mount('/content/drive')
path = "/content/drive/Shared drives/PythonCoaching_2023Summer"
my_folder = "ljw33083418"
outcome_folder = f"{path}/{my_folder}/outcome/project/htmls"

for key, value in categories.items():

  r_category = requests.get(value)
  soup_category = BeautifulSoup(r_category.content, "html.parser")
  navigation = soup_category.find("div", {"class": "post-navigation"}).find('ul').find_all('li')
  max_pagenum = int(navigation[-2].find('a').text)

  file_name = key + '_'

  if key == 'Web':
    for i in range(623, max_pagenum+1): #해당 category에 대한 모든 page

      r_page = requests.get(value + "page/" + str(i))
      soup_page = BeautifulSoup(r_page.content, "html.parser")

      articles = soup_page.find_all("h2", {"class": "entry-title"}) #여기서 앞의 20개 article들은 most popular article들로, page에 관계 없이 계속 반복됨
      articles = articles[20:]

      for article in articles:
        try:
          article = article.find('a')
          title = article.text; url = article["href"]

          r_article = requests.get(url)

          file_name += url[len("https://readwrite.com/"):-1] + ".html"
          with open(f"{outcome_folder}/{file_name}", "w+b") as fw:
              fw.write(r_article.content)
          file_name = key + '_'
        except Exception as e:
          print(f"An error occurred for {url}: {e}")
        time.sleep(1)

  else:
    for i in range(1, max_pagenum+1): #해당 category에 대한 모든 page

      r_page = requests.get(value + "page/" + str(i))
      soup_page = BeautifulSoup(r_page.content, "html.parser")

      articles = soup_page.find_all("h2", {"class": "entry-title"}) #여기서 앞의 20개 article들은 most popular article들로, page에 관계 없이 계속 반복됨
      articles = articles[20:]

      for article in articles:
        try:
          article = article.find('a')
          title = article.text; url = article["href"]

          r_article = requests.get(url)

          file_name += url[len("https://readwrite.com/"):-1] + ".html"
          with open(f"{outcome_folder}/{file_name}", "w+b") as fw:
              fw.write(r_article.content)
          file_name = key + '_'
        except Exception as e:
          print(f"An error occurred for {url}: {e}")
        time.sleep(1)

In [ ]:
# 정규표현식 테스트
import re

input_string = "This is a test string with extra spaces. author: John Doe"
author = "John Doe"
pattern = re.escape(author)
output_string = re.sub(pattern, '', input_string)
print(output_string)

In [ ]:
# html files read

from google.colab import drive
import os
import re

drive.mount('/content/drive')
path = "/content/drive/Shared drives/PythonCoaching_2023Summer"
my_folder = "ljw33083418"
outcome_folder = f"{path}/{my_folder}/outcome/project/"

html_files = [file for file in os.listdir(f"{outcome_folder}/htmls") if file.endswith(".html")]
html_files = list(set(html_files))
html_files.sort()
total_files_length = len(html_files) #28245

In [ ]:
# html_data.csv 생성

with open(f"{outcome_folder}/html_data.csv", "a") as fw:
  #fw.write("category\ttitle\tauthor\ttimestamp\tbody\n")
  for i in range(4727, total_files_length):
    file_name = html_files[i]
    with open(f"{outcome_folder}/htmls/{file_name}", "r+b") as fr:
      soup = BeautifulSoup(fr.read(), "html.parser")

      categories = set()
      origin_cat = file_name.split('_')[0]
      if origin_cat == 'AR_VR':
        origin_cat = 'AR/VR'
      categories.add(origin_cat)

      category_hrefs = soup.find("span", {"class": "author catname"})
      if category_hrefs:
        category_hrefs = category_hrefs.find_all("a")
      if category_hrefs:
        for href in category_hrefs:
          if href.text == 'ReadWrite':
            continue
          categories.add(href.text)

      title = soup.find("h1", {"class": "entry-title"})
      if title: title = title.text
      else: continue

      author = soup.find("span", {"class": "author"})
      if author: author = author.find('a').text.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200a', ' ').strip()
      else: continue

      timestamp = soup.find("span", {"class": "post-cat"})
      if timestamp: timestamp = timestamp.text.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200a', ' ').strip()
      else: continue

      body = soup.find("div", {"class": "entry-content col-md-10"})
      if body: body = body.text.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200a', ' ').strip()
      else: continue

      match_index = re.search(author, body)
      if match_index:
        start_index = match_index.start()
        body = body[:start_index]
      body = body.strip()


      if categories and title and author and timestamp and body:
        fw.write(f"{categories}\t{title}\t{author}\t{timestamp}\t{body}\n")

      print(str(i)+'/'+str(total_files_length))

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/Shared drives/PythonCoaching_2023Summer"
my_folder = "ljw33083418"
outcome_folder = f"{path}/{my_folder}/outcome/project/"

pd.set_option('display.max_colwidth', 150)

df_who = pd.read_csv(f"{outcome_folder}/html_data.csv", sep="\t", lineterminator='\n')
df_who

In [ ]:
df_who.info()